[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/CMU-IDeeL/CMU-IDeeL.github.io/blob/master/F25/document/Recitation_0_Series/0.5/0_5_Tensordot_&_Einsum.ipynb)

 # 0.5 Tensordot & Einsum

 ## Authors:


*   Ahmed Alhassan
*   Michael Kireeff





 This notebook explores `einsum` (Einstein summation) and `tensordot`, two powerful functions for tensor operations. They offer a concise way to do dot products, outer products, transpositions, matrix math, and more complex tensor contractions.

 ## Mathematical Foundation





### What is a Tensor?

Tensors are multi-dimensional arrays capable of representing data from scalars (0D) to vectors (1D), matrices (2D), and higher-dimensional structures. Both PyTorch and Numpy provide a wide range of tensor operations.

### Tensor Operations

It is the set of mathematical operations that involve the use of tensors, those operations include:

*   Mathematical transformations: Addition, multiplication, and advanced matrix computations
*   Shape manipulation: Reshaping, stacking, and squeezing dimensions
*   Data extraction: Indexing, slicing, and conditional filtering
*   Reduction operations: Summing, averaging, and finding max/min values
*   GPU-accelerated processing: Harnessing parallel computing power
*   Automatic differentiation: Calculating gradients for machine learning

### Einstein Summation Convention

 The Einstein summation convention is a notational convention that implies summation over repeated indices.

 For example, if we have tensors $A_{ij}$ and $B_{jk}$, then $C_{ik} = A_{ij} B_{jk}$ represents:



 $C_{ik} = \sum_j A_{ij} B_{jk}$



 This convention eliminates the need to explicitly write summation symbols, making tensor operations more concise.



 ### Tensordot (Tensor Contraction)

 Tensor contraction is a generalization of matrix multiplication to higher-dimensional arrays.

 When we contract along certain dimensions, we perform element-wise multiplication followed by summation

 along those dimensions.

 ## Imports

In [ ]:
import numpy as np
import torch
import time


 ## Einsum



 **Einstein summation** (`einsum`) is a super handy function in NumPy and PyTorch. It uses a special string notation (Einstein's convention) to perform a wide variety of tensor operations. 💡



 The basic idea is `einsum('input_specs -> output_spec', tensor1, tensor2, ...)`.

 - Indices repeated in inputs but not in the output get summed out (contracted).

 - Indices appearing in an input but not the output are also summed out.

 - The order of indices in the `output_spec` defines the final tensor's axis order.

 - If `-> output_spec` is missing, `einsum` infers it from unsummed input indices, ordered alphabetically.

 ```numpy.einsum(subscripts, *operands, out=None, dtype=None, order='K', casting='safe')```



 ```torch.einsum(equation, *operands)```

 ### Vector Operations



 Let's start with some basic vector ops.

In [ ]:
u_vec_ops = np.array([1, 2, 3])
v_vec_ops = np.array([4, 5, 6])
print("u_vec_ops:", u_vec_ops, "Shape:", u_vec_ops.shape)
print("v_vec_ops:", v_vec_ops, "Shape:", v_vec_ops.shape)


u_vec_ops: [1 2 3] Shape: (3,)
v_vec_ops: [4 5 6] Shape: (3,)


 #### Sum of Elements (Vector Sum Reduction)

This operation calculates the sum of all elements in a vector.

 Equation: $s = \sum_i u_i$.



 For `einsum`:

 -  `'i->'`:

   -  `i`: A single 1D input indexed by `i`.

   -  `->`: No output index, so `i` is summed over.

In [ ]:
sum_elements_einsum = np.einsum('i->', u_vec_ops)
sum_elements_expected = np.sum(u_vec_ops)
print("Sum of elements in u_vec_ops (einsum):", sum_elements_einsum)
print("Expected (np.sum):", sum_elements_expected)
print("Results match:", np.allclose(sum_elements_einsum, sum_elements_expected))


Sum of elements in u_vec_ops (einsum): 6
Expected (np.sum): 6
Results match: True


 #### Element-wise Vector Product (Hadamard Product)

 This is the **Hadamard product**, meaning element-by-element multiplication.

 If $w = u \odot v$, then each output element is $w_i = u_i \cdot v_i$.



 For `einsum`'s `subscripts`:

 -  `'i,i->i'`:

   -  `i,i`: Two 1D inputs, both indexed by `i`.

   -  `->i`: Output is 1D, also indexed by `i`.

   -  Since `i` is in both inputs and the output, it's an element-wise operation. No summation occurs over `i`.

In [ ]:
hadamard_einsum = np.einsum('i,i->i', u_vec_ops, v_vec_ops)
hadamard_expected = np.multiply(u_vec_ops, v_vec_ops)
print("Result (einsum):", hadamard_einsum)
print("Expected (np.multiply):", hadamard_expected)
print("Results match:", np.allclose(hadamard_einsum, hadamard_expected))


Result (einsum): [ 4 10 18]
Expected (np.multiply): [ 4 10 18]
Results match: True


 #### Dot Product (Inner Product)

 The **dot product** (or scalar product) sums the products of corresponding vector elements, yielding a single scalar value.

 The equation is $s = u \cdot v = \sum_i u_i v_i$.



 For `einsum`:

 -  `'i,i->'`:

   -  `i,i`: Two 1D inputs, indexed by `i`.

   -  `->`: No output index means `i` (which is common to inputs but not output) is summed over.

 -  `'i,i'`: This is shorthand; if `->` is omitted, repeated indices not in an explicit output are summed.

In [ ]:
dot_explicit = np.einsum('i,i->', u_vec_ops, v_vec_ops)
dot_implicit = np.einsum('i,i', u_vec_ops, v_vec_ops)
dot_expected = np.dot(u_vec_ops, v_vec_ops)

print("Result (explicit output 'i,i->'):", dot_explicit)
print("Result (implicit sum 'i,i'):", dot_implicit)
print("Expected (np.dot):", dot_expected)
print("All results match:", np.allclose([dot_explicit, dot_implicit, dot_expected], [dot_expected]*3))

# Performance comparison (actual %timeit commented out for brevity in output)
print("\nPerformance Comparison:")
print("Einsum timing for dot product:")
%timeit np.einsum('i,i->', u_vec_ops, v_vec_ops)
print("NumPy dot timing for dot product:")
%timeit np.dot(u_vec_ops, v_vec_ops)


Result (explicit output 'i,i->'): 32
Result (implicit sum 'i,i'): 32
Expected (np.dot): 32
All results match: True

Performance Comparison:
Einsum timing for dot product:
2.07 µs ± 287 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
NumPy dot timing for dot product:
861 ns ± 15.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


 #### Outer Product





The **outer product** of two vectors $u$ (size $m$) and $v$ (size $n$) results in an $m \times n$ matrix $M$.

 Each element $M_{ij} = u_i v_j$.



 For `einsum`:

 -  `'i,j->ij'`:

   -  `i,j`: First input indexed by `i`, second by `j`.

   -  `->ij`: Output is 2D, with axes corresponding to `i` and `j`. No summation as `i` and `j` are distinct and present in the output.

In [ ]:
outer_einsum = np.einsum('i,j->ij', u_vec_ops, v_vec_ops)
outer_expected = np.outer(u_vec_ops, v_vec_ops)
print("Outer product (einsum):\n", outer_einsum)
print("Expected (np.outer):\n", outer_expected)
print("Results match:", np.allclose(outer_einsum, outer_expected))


Outer product (einsum):
 [[ 4  5  6]
 [ 8 10 12]
 [12 15 18]]
Expected (np.outer):
 [[ 4  5  6]
 [ 8 10 12]
 [12 15 18]]
Results match: True


 ### Matrix Operations



 Now, let's work with matrices.

In [ ]:
A_mat = np.array([[1, 2, 3], [4, 5, 6]]) # Shape (2,3)
B_mat = np.array([[1, 2], [3, 4], [5, 6]]) # Shape (3,2)
C_mat = np.array([[1, 2], [3, 4]])     # Shape (2,2)
# u_vec_ops is (3,) - can be used as u_matvec
# w_matvec needs to be defined if used for vector-matrix product with C_mat, or matrix-matrix-vector product
w_matvec = np.array([7, 8]) # Shape (2,) for (A_mat @ B_mat) @ w_matvec or w_matvec @ C_mat

print("A_mat (2x3):\n", A_mat)
print("B_mat (3x2):\n", B_mat)
print("C_mat (2x2):\n", C_mat)
print("u_vec_ops (3,):\n", u_vec_ops) # Reusing u_vec_ops for matrix-vector product
print("w_matvec (2,):\n", w_matvec)



A_mat (2x3):
 [[1 2 3]
 [4 5 6]]
B_mat (3x2):
 [[1 2]
 [3 4]
 [5 6]]
C_mat (2x2):
 [[1 2]
 [3 4]]
u_vec_ops (3,):
 [1 2 3]
w_matvec (2,):
 [7 8]


 #### Matrix Transpose





The **transpose** of a matrix $M$ (denoted $M^T$) swaps its rows and columns. If $M$ is $m \times n$, $M^T$ is $n \times m$.

 Equation: $(M^T)_{ji} = M_{ij}$.



 For `einsum`:

 -  `'ij->ji'`:

   -  `ij`: Input is 2D (rows `i`, columns `j`).

   -  `->ji`: Output is 2D, with original rows `i` now columns, and original columns `j` now rows.

In [ ]:
transpose_einsum = np.einsum('ij->ji', A_mat)
transpose_expected = A_mat.T
print("Transpose of A_mat (einsum):\n", transpose_einsum)
print("Expected (A_mat.T):\n", transpose_expected)
print("Results match:", np.allclose(transpose_einsum, transpose_expected))


Transpose of A_mat (einsum):
 [[1 4]
 [2 5]
 [3 6]]
Expected (A_mat.T):
 [[1 4]
 [2 5]
 [3 6]]
Results match: True


 #### Trace of a Matrix





The **trace** of a square matrix $M$ (denoted $\text{tr}(M)$) is the sum of elements on its main diagonal.

 Equation: $\text{tr}(M) = \sum_i M_{ii}$.



 For `einsum`:

 -  `'ii->'`:

   -  `ii`: Input is 2D. Using the same index `i` for rows and columns selects diagonal elements.

   -  `->`: No output index, so these diagonal elements are summed.

In [ ]:
trace_einsum = np.einsum('ii->', C_mat) # C_mat is 2x2
trace_expected = np.trace(C_mat)
print("Trace of C_mat (einsum 'ii->'):", trace_einsum)
print("Expected (np.trace):", trace_expected)
print("Results match:", np.allclose(trace_einsum, trace_expected))


Trace of C_mat (einsum 'ii->'): 5
Expected (np.trace): 5
Results match: True


 #### Diagonal Extraction





Extracts the main diagonal of a square matrix $M$ into a vector.

 Equation: $d_i = M_{ii}$.



 For `einsum`:

 -  `'ii->i'`:

   -  `ii`: Selects diagonal elements.

   -  `->i`: Output is 1D, containing these elements.

In [ ]:
diag_extract_einsum = np.einsum('ii->i', C_mat)
diag_extract_expected = np.diag(C_mat)
print("Diagonal of C_mat (einsum 'ii->i'):", diag_extract_einsum)
print("Expected (np.diag):", diag_extract_expected)
print("Results match:", np.allclose(diag_extract_einsum, diag_extract_expected))


Diagonal of C_mat (einsum 'ii->i'): [1 4]
Expected (np.diag): [1 4]
Results match: True


 #### Sum along an Axis





1. **Sum rows (collapse columns, sum over `j` for each `i`):** For $M_{ij}$, $r_i = \sum_j M_{ij}$.

   -  `'ij->i'`: Index `j` (columns) is summed out.

 2. **Sum columns (collapse rows, sum over `i` for each `j`):** For $M_{ij}$, $c_j = \sum_i M_{ij}$.

   -  `'ij->j'`: Index `i` (rows) is summed out.

In [ ]:
print("Matrix A_mat:\n", A_mat)
sum_along_axis1_einsum = np.einsum('ij->i', A_mat) # Sum over columns for each row
sum_along_axis1_expected = np.sum(A_mat, axis=1)
print("Sum rows of A_mat (sum over columns for each row, einsum 'ij->i'):", sum_along_axis1_einsum)
print("Expected (np.sum(A_mat, axis=1)):", sum_along_axis1_expected)
print("Results match:", np.allclose(sum_along_axis1_einsum, sum_along_axis1_expected))

sum_along_axis0_einsum = np.einsum('ij->j', A_mat) # Sum over rows for each column
sum_along_axis0_expected = np.sum(A_mat, axis=0)
print("\nSum columns of A_mat (sum over rows for each column, einsum 'ij->j'):", sum_along_axis0_einsum)
print("Expected (np.sum(A_mat, axis=0)):", sum_along_axis0_expected)
print("Results match:", np.allclose(sum_along_axis0_einsum, sum_along_axis0_expected))


Matrix A_mat:
 [[1 2 3]
 [4 5 6]]
Sum rows of A_mat (sum over columns for each row, einsum 'ij->i'): [ 6 15]
Expected (np.sum(A_mat, axis=1)): [ 6 15]
Results match: True

Sum columns of A_mat (sum over rows for each column, einsum 'ij->j'): [5 7 9]
Expected (np.sum(A_mat, axis=0)): [5 7 9]
Results match: True


 #### Frobenius Norm (Matrix Norm)





 Mathematical notation: $||A||_F = \sqrt{\sum_i \sum_j A_{ij}^2}$

 Einsum can compute the squared norm: $\sum_i \sum_j A_{ij}^2 = \sum_i \sum_j A_{ij} A_{ij}$

 Einsum notation: `'ij,ij->'` for the squared norm.

In [ ]:
# Using A_mat (2x3) for Frobenius norm
frobenius_sq_einsum = np.einsum('ij,ij->', A_mat, A_mat)
frobenius_einsum = np.sqrt(frobenius_sq_einsum)
frobenius_expected = np.linalg.norm(A_mat, 'fro')

print(f"A_mat for Frobenius norm:\n{A_mat}")
print("Squared Frobenius norm (einsum 'ij,ij->'):", frobenius_sq_einsum)
print("Frobenius norm (einsum, sqrt):", frobenius_einsum)
print("Expected (np.linalg.norm):", frobenius_expected)
print("Results match:", np.allclose(frobenius_einsum, frobenius_expected))


A_mat for Frobenius norm:
[[1 2 3]
 [4 5 6]]
Squared Frobenius norm (einsum 'ij,ij->'): 91
Frobenius norm (einsum, sqrt): 9.539392014169456
Expected (np.linalg.norm): 9.539392014169456
Results match: True


 #### Element-wise Matrix Product (Hadamard)



For two matrices $M_1, M_2$ of same dimensions, $P = M_1 \odot M_2 \implies P_{ij} = (M_1)_{ij} (M_2)_{ij}$.

 -  `'ij,ij->ij'`: Common indices `i,j` in inputs and output means element-wise.

In [ ]:
D_mat = np.array([[5, 6], [7, 8]]) # Shape (2,2), same as C_mat
hadamard_mat_einsum = np.einsum('ij,ij->ij', C_mat, D_mat)
hadamard_mat_expected = C_mat * D_mat # or np.multiply(C_mat, D_mat)
print("Element-wise product C_mat * D_mat (einsum 'ij,ij->ij'):\n", hadamard_mat_einsum)
print("Expected (C_mat * D_mat):\n", hadamard_mat_expected)
print("Results match:", np.allclose(hadamard_mat_einsum, hadamard_mat_expected))


Element-wise product C_mat * D_mat (einsum 'ij,ij->ij'):
 [[ 5 12]
 [21 32]]
Expected (C_mat * D_mat):
 [[ 5 12]
 [21 32]]
Results match: True


 #### Matrix-Vector Product





Product of an $m \times n$ matrix $M$ and an $n \times 1$ (column) vector $u$, resulting in an $m \times 1$ vector $y$.

 Equation: $y = M u \implies y_i = \sum_j M_{ij} u_j$.



 For `einsum`:

 -  `'ij,j->i'`:

   -  `ij,j`: Matrix $M$ (indices `i,j`) and vector $u$ (index `j`).

   -  `->i`: Output is 1D (index `i`).

   -  Index `j` is common to inputs but not output, so it's summed. Index `i` (rows of $M$) is preserved.

In [ ]:
matvec_einsum = np.einsum('ij,j->i', A_mat, u_vec_ops) # Using u_vec_ops as the vector
matvec_expected = np.dot(A_mat, u_vec_ops) # or A_mat @ u_vec_ops
print("Matrix-vector product A_mat @ u_vec_ops (einsum):\n", matvec_einsum)
print("Expected (np.dot):\n", matvec_expected)
print("Results match:", np.allclose(matvec_einsum, matvec_expected))

# Step-by-step calculation for understanding:
print("\nStep-by-step calculation for A_mat @ u_vec_ops:")
print(f"A_mat[0,:] ({A_mat[0,:]}) @ u_vec_ops ({u_vec_ops}) = {np.dot(A_mat[0,:], u_vec_ops)}")
print(f"A_mat[1,:] ({A_mat[1,:]}) @ u_vec_ops ({u_vec_ops}) = {np.dot(A_mat[1,:], u_vec_ops)}")


Matrix-vector product A_mat @ u_vec_ops (einsum):
 [14 32]
Expected (np.dot):
 [14 32]
Results match: True

Step-by-step calculation for A_mat @ u_vec_ops:
A_mat[0,:] ([1 2 3]) @ u_vec_ops ([1 2 3]) = 14
A_mat[1,:] ([4 5 6]) @ u_vec_ops ([1 2 3]) = 32


 #### Vector-Matrix Product





Product of a $1 \times m$ (row) vector $w^T$ and an $m \times n$ matrix $M$, resulting in a $1 \times n$ vector $z^T$.

 (Represent $w^T$ as a 1D array `w_arr`).

 Equation: $z^T = w^T M \implies z_j = \sum_i w_i M_{ij}$.



 For `einsum`:

 -  `'i,ij->j'`:

   -  `i,ij`: Vector `w_arr` (index `i`) and matrix $M$ (indices `i,j`).

   -  `->j`: Output is 1D (index `j`).

   -  Index `i` is summed. Index `j` (columns of $M$) is preserved.

In [ ]:
# Using w_matvec (shape 2) and C_mat (shape 2x2)
vecmat_einsum = np.einsum('i,ij->j', w_matvec, C_mat)
vecmat_expected = np.dot(w_matvec, C_mat) # or w_matvec @ C_mat
print("Vector-matrix product w_matvec @ C_mat (einsum):\n", vecmat_einsum)
print("Expected (np.dot):\n", vecmat_expected)
print("Results match:", np.allclose(vecmat_einsum, vecmat_expected))


Vector-matrix product w_matvec @ C_mat (einsum):
 [31 46]
Expected (np.dot):
 [31 46]
Results match: True


 #### Matrix Multiplication





Product of an $m \times n$ matrix $M_1$ and an $n \times p$ matrix $M_2$, results in an $m \times p$ matrix $P$.

 Equation: $P = M_1 M_2 \implies P_{ik} = \sum_j (M_1)_{ij} (M_2)_{jk}$.



 For `einsum`:

 -  `'ij,jk->ik'`:

   -  `ij,jk`: Matrix $M_1$ (`i,j`) and $M_2$ (`j,k`).

   -  `->ik`: Output is 2D (`i,k`).

   -  `j` is summed. `i` (rows of $M_1$) and `k` (columns of $M_2$) define output shape.

 -  `'ij,jk'`: Shorthand, `ik` output is inferred.

In [ ]:
matmul_einsum_explicit = np.einsum('ij,jk->ik', A_mat, B_mat)
matmul_einsum_implicit = np.einsum('ij,jk', A_mat, B_mat)
matmul_expected = np.matmul(A_mat, B_mat) # or A_mat @ B_mat
print("Matrix multiplication A_mat @ B_mat (einsum, explicit 'ij,jk->ik'):\n", matmul_einsum_explicit)
print("Matrix multiplication A_mat @ B_mat (einsum, implicit 'ij,jk'):\n", matmul_einsum_implicit)
print("Expected (np.matmul):\n", matmul_expected)
print("All results match:", np.allclose(matmul_einsum_explicit, matmul_expected) and np.allclose(matmul_einsum_implicit, matmul_expected))


Matrix multiplication A_mat @ B_mat (einsum, explicit 'ij,jk->ik'):
 [[22 28]
 [49 64]]
Matrix multiplication A_mat @ B_mat (einsum, implicit 'ij,jk'):
 [[22 28]
 [49 64]]
Expected (np.matmul):
 [[22 28]
 [49 64]]
All results match: True


 #### Matrix Multiplication with Reduction





Performs $M_1 M_2 = P$, then sums $P$ over one or both axes.

 1. **Sum columns of $P$ (sum each row of $P$):** $r_i = \sum_k P_{ik} = \sum_k \sum_j (M_1)_{ij} (M_2)_{jk}$.

   -  `'ij,jk->i'`: Index `k` from $(M_2)_{jk}$ is summed out.

 2. **Sum rows of $P$ (sum each column of $P$):** $s_k = \sum_i P_{ik} = \sum_i \sum_j (M_1)_{ij} (M_2)_{jk}$.

   -  `'ij,jk->k'`: Index `i` from $(M_1)_{ij}$ is summed out.

 3. **Sum all elements of $P$:** $t = \sum_i \sum_k P_{ik}$.

   -  `'ij,jk->'`: Both `i` and `k` are summed out.

In [ ]:
P_product = A_mat @ B_mat # Result is (2x2)

# Sum over columns of P_product (sum each row)
sum_cols_einsum = np.einsum('ij,jk->i', A_mat, B_mat)
sum_cols_expected = np.sum(P_product, axis=1)
print("A_mat @ B_mat then sum over columns (einsum 'ij,jk->i'):\n", sum_cols_einsum)
print("Expected (sum rows of P_product):\n", sum_cols_expected)
print("Results match:", np.allclose(sum_cols_einsum, sum_cols_expected))

# Sum over rows of P_product (sum each column)
sum_rows_einsum = np.einsum('ij,jk->k', A_mat, B_mat)
sum_rows_expected = np.sum(P_product, axis=0)
print("\nA_mat @ B_mat then sum over rows (einsum 'ij,jk->k'):\n", sum_rows_einsum)
print("Expected (sum cols of P_product):\n", sum_rows_expected)
print("Results match:", np.allclose(sum_rows_einsum, sum_rows_expected))

# Sum all elements of P_product
sum_all_einsum = np.einsum('ij,jk->', A_mat, B_mat)
sum_all_expected = np.sum(P_product)
print("\nA_mat @ B_mat then sum all elements (einsum 'ij,jk->'):\n", sum_all_einsum)
print("Expected (sum all elements of P_product):\n", sum_all_expected)
print("Results match:", np.allclose(sum_all_einsum, sum_all_expected))


A_mat @ B_mat then sum over columns (einsum 'ij,jk->i'):
 [ 50 113]
Expected (sum rows of P_product):
 [ 50 113]
Results match: True

A_mat @ B_mat then sum over rows (einsum 'ij,jk->k'):
 [71 92]
Expected (sum cols of P_product):
 [71 92]
Results match: True

A_mat @ B_mat then sum all elements (einsum 'ij,jk->'):
 163
Expected (sum all elements of P_product):
 163
Results match: True


 #### Combination of Operations (Chained Operations)





 `einsum` can chain multiple operations, e.g., $(M_1 M_2)w$.



 1. **$(M_1 M_2)w \rightarrow \text{vector}$**:

   $M_1 (m \times n), M_2 (n \times p), w (p \times 1)$.

   Equation: $r_i = \sum_k \left( \sum_j (M_1)_{ij} (M_2)_{jk} \right) w_k$.

   -  `'ij,jk,k->i'`: `j` and `k` are summed out.



 2. **$\text{sum}((M_1 M_2)w) \rightarrow \text{scalar}$**:

   Equation: $s = \sum_i \sum_k \left( \sum_j (M_1)_{ij} (M_2)_{jk} \right) w_k$.

   -  `'ij,jk,k->'`: `i, j, k` are summed out.

In [ ]:
# A_mat (2,3), B_mat (3,2), w_matvec (2,)
# (A_mat @ B_mat) is (2,2). (A_mat @ B_mat) @ w_matvec is (2,)
chain1_einsum = np.einsum('ij,jk,k->i', A_mat, B_mat, w_matvec)
chain1_expected = (A_mat @ B_mat) @ w_matvec
print("Result of (A_mat B_mat) w_matvec (einsum 'ij,jk,k->i'):", chain1_einsum)
print("Expected ((A_mat@B_mat) @ w_matvec):\n", chain1_expected)
print("Results match:", np.allclose(chain1_einsum, chain1_expected))

chain2_einsum = np.einsum('ij,jk,k->', A_mat, B_mat, w_matvec)
chain2_expected = np.sum((A_mat @ B_mat) @ w_matvec)
print("\nResult of sum((A_mat B_mat) w_matvec) (einsum 'ij,jk,k->'):", chain2_einsum)
print("Expected (sum of ((A_mat@B_mat) @ w_matvec)):\n", chain2_expected)
print("Results match:", np.allclose(chain2_einsum, chain2_expected))


Result of (A_mat B_mat) w_matvec (einsum 'ij,jk,k->i'): [378 855]
Expected ((A_mat@B_mat) @ w_matvec):
 [378 855]
Results match: True

Result of sum((A_mat B_mat) w_matvec) (einsum 'ij,jk,k->'): 1233
Expected (sum of ((A_mat@B_mat) @ w_matvec)):
 1233
Results match: True


 3. **$M_1 M_2 M_3 \rightarrow \text{matrix}$**:

   $M_1 (m \times n), M_2 (n \times p), M_3 (p \times q)$.

   Equation: $R_{il} = \sum_k \left( \sum_j (M_1)_{ij} (M_2)_{jk} \right) (M_3)_{kl}$.

   -  `'ij,jk,kl->il'`: `j` and `k` are summed out.

In [ ]:
# A_mat (2,3), B_mat (3,2), C_mat (2,2). (A_mat@B_mat@C_mat) is (2,2).
chain3_einsum = np.einsum('ij,jk,kl->il', A_mat, B_mat, C_mat)
chain3_expected = A_mat @ B_mat @ C_mat
print("\nResult of A_mat B_mat C_mat (A_mat@B_mat@C_mat) (einsum 'ij,jk,kl->il'):\n", chain3_einsum)
print("Expected (A_mat@B_mat@C_mat):\n", chain3_expected)
print("Results match:", np.allclose(chain3_einsum, chain3_expected))



Result of A_mat B_mat C_mat (A_mat@B_mat@C_mat) (einsum 'ij,jk,kl->il'):
 [[106 156]
 [241 354]]
Expected (A_mat@B_mat@C_mat):
 [[106 156]
 [241 354]]
Results match: True


 ### Tensor Operations with Einsum



 `einsum` really shines with tensors of rank 3+.

In [ ]:
np.random.seed(42) # For reproducibility
Tensor_A_orig = np.random.rand(2, 3, 4, 5) # b, c, h, w for example
Tensor_B_orig = np.random.rand(2, 3, 5, 6) # b, c, w, d for example
print("Dimensions of Tensor_A_orig:", Tensor_A_orig.shape)
print("Dimensions of Tensor_B_orig:", Tensor_B_orig.shape)


Dimensions of Tensor_A_orig: (2, 3, 4, 5)
Dimensions of Tensor_B_orig: (2, 3, 5, 6)


 #### "..." Operator (Ellipsis)





The ellipsis (`...`) is a placeholder for any number of leading dimensions. It's great for batch operations without naming all leading dimensions.



 Example: Batch matrix multiplication. If `T_A` is `(..., m, n)` and `T_B` is `(..., n, p)`, then `einsum('...ij,...jk->...ik', T_A, T_B)` does matrix multiplication for each slice in `...`, outputting `(..., m, p)`. The `...` dimensions must be broadcastable.



 For `Tensor_A_orig (2,3,4,5)` and `Tensor_B_orig (2,3,5,6)`:

 If we treat `(2,3)` as batch dimensions (`...`):

 - `...` maps to `(2,3)`.

 - `ij` maps to `(4,5)` for `Tensor_A_orig`.

 - `jk` maps to `(5,6)` for `Tensor_B_orig`. (Here `j` is the common dimension of size 5)

 - Result `...ik` has shape `(2,3,4,6)`.

In [ ]:
A_batch_mm = np.random.rand(2,3,4,5) # Batch of 4x5 matrices
B_batch_mm = np.random.rand(2,3,5,6) # Batch of 5x6 matrices

result_ellipsis = np.einsum('...ij,...jk->...ik', A_batch_mm, B_batch_mm)
print("Dimensions of result_ellipsis ('...ij,...jk->...ik'):", result_ellipsis.shape) # Expected (2,3,4,6)
result_matmul = np.matmul(A_batch_mm, B_batch_mm)
print("Dimensions of np.matmul result:", result_matmul.shape)
print("Ellipsis and matmul results match:", np.allclose(result_ellipsis, result_matmul))

result_ellipsis_orig = np.einsum('...ij,...jk->...ik', Tensor_A_orig, Tensor_B_orig)
print("\nDimensions of the result using ellipsis ('...ij,...jk->...ik') on original Tensors:", result_ellipsis_orig.shape)

# Performance comparison (actual %timeit commented out)
print("\nPerformance comparison for batch matmul:")
print("Einsum with ellipsis timing:")
%timeit np.einsum('...ij,...jk->...ik', A_batch_mm, B_batch_mm)
print("NumPy matmul timing:")
%timeit np.matmul(A_batch_mm, B_batch_mm)

print("\nShape of np.dot(Tensor_A_orig, Tensor_B_orig):", np.dot(Tensor_A_orig, Tensor_B_orig).shape)



Dimensions of result_ellipsis ('...ij,...jk->...ik'): (2, 3, 4, 6)
Dimensions of np.matmul result: (2, 3, 4, 6)
Ellipsis and matmul results match: True

Dimensions of the result using ellipsis ('...ij,...jk->...ik') on original Tensors: (2, 3, 4, 6)

Performance comparison for batch matmul:
Einsum with ellipsis timing:
6.5 µs ± 1.69 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)
NumPy matmul timing:
2.29 µs ± 47 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)

Shape of np.dot(Tensor_A_orig, Tensor_B_orig): (2, 3, 4, 2, 3, 6)


 #### Tensor Dot (Specific Contraction)





A specific tensor contraction. For `T_A (i,j,k,l)` and `T_B (i,j,l,m)`.

 `einsum('ijkl,ijlm->km', T_A, T_B)`



 -  Inputs: `Tensor_A_orig (2,3,4,5)` and `Tensor_B_orig (2,3,5,6)`.

   -  For `Tensor_A_orig`: `i,j,k,l` map to axes `0,1,2,3`.

   -  For `Tensor_B_orig`: `i,j,l,m` maps to axes `0,1,2,3` (so `Tensor_B_orig`'s axis 2 is size 5, axis 3 is size 6).

 -  Output: 2D tensor (indices `k,m`).

 -  Indices `i,j,l` are summed over (axis 0 of A/B; axis 1 of A/B; axis 3 of A / axis 2 of B).

 -  Result shape is `(Tensor_A_orig.shape[2], Tensor_B_orig.shape[3])` = `(4,6)`.



 Equation: $R_{km} = \sum_i \sum_j \sum_l (T_A)_{ijkl} (T_B)_{ijlm}$.

In [ ]:
result_tensor_dot = np.einsum('ijkl,ijlm->km', Tensor_A_orig, Tensor_B_orig)
print("Dimensions of 'ijkl,ijlm->km' result:", result_tensor_dot.shape) # Expected (4,6)

# Example from file 2: More complex contraction
# A_4d (2,3,4,5), B_4d (2,3,5,6) - same as Tensor_A_orig, Tensor_B_orig
complex_contraction = np.einsum('ijkl,mjln->ikmn', Tensor_A_orig, Tensor_B_orig) # Example: T_A(i,j,k,l) T_B(m,j,k,n) -> T_out(i,l,m,n)
print("Dimensions of 'ijkl,mjln->ikmn' result:", complex_contraction.shape) # Expected (4,6)

complex_example = np.einsum('abci,abjd->cijd', Tensor_A_orig, Tensor_B_orig)
print("Complex contraction 'abci,abjd->cijd' result shape:", complex_example.shape)



Dimensions of 'ijkl,ijlm->km' result: (4, 6)
Dimensions of 'ijkl,mjln->ikmn' result: (2, 4, 2, 6)
Complex contraction 'abci,abjd->cijd' result shape: (4, 5, 5, 6)


 #### Batch Matrix Multiplication (Explicit Batch Index)





For 3D tensors $T_A (b, r_A, c_A)$ and $T_B (b, c_A, c_B)$, perform matrix multiplication for each batch $b$.

 Equation: $(P_b)_{ik} = \sum_j (T_A)_{bij} (T_B)_{bjk}$.

 -  `'bij,bjk->bik'`: `b` is batch, `j` is summed.

In [ ]:
Batch_A = np.random.rand(10, 4, 5) # 10 batches of 4x5 matrices
Batch_B = np.random.rand(10, 5, 6) # 10 batches of 5x6 matrices

result_batch_mm_explicit = np.einsum('bij,bjk->bik', Batch_A, Batch_B)
print("Batch MatMul ('bij,bjk->bik') result shape:", result_batch_mm_explicit.shape) # Expected (10,4,6)
result_batch_mm_matmul = np.matmul(Batch_A, Batch_B)
print("Batch MatMul (np.matmul) result shape:", result_batch_mm_matmul.shape)
print("Results match:", np.allclose(result_batch_mm_explicit, result_batch_mm_matmul))



Batch MatMul ('bij,bjk->bik') result shape: (10, 4, 6)
Batch MatMul (np.matmul) result shape: (10, 4, 6)
Results match: True


 #### Permutation of Tensor Axes





Reorders tensor axes. E.g., $T_{ijk} \rightarrow T'_{kij}$.

 Equation: $T'_{kij} = T_{ijk}$.

 -  `'ijk->kij'`: Axes `i,j,k` are reordered to `k,i,j`.

In [ ]:
Tensor_permute = np.random.rand(2,3,4) # Indices i,j,k
print("Original Tensor_permute shape (ijk):", Tensor_permute.shape)
permuted_T_einsum = np.einsum('ijk->kij', Tensor_permute)
permuted_T_expected = np.transpose(Tensor_permute, axes=(2,0,1))
print("Permuted Tensor_permute shape (kij) (einsum 'ijk->kij'):", permuted_T_einsum.shape) # Expected (4,2,3)
print("Permuted Tensor_permute shape (kij) (np.transpose(..., axes=(2,0,1))):", permuted_T_expected.shape)
print("Results match:", np.array_equal(permuted_T_einsum, permuted_T_expected))


Original Tensor_permute shape (ijk): (2, 3, 4)
Permuted Tensor_permute shape (kij) (einsum 'ijk->kij'): (4, 2, 3)
Permuted Tensor_permute shape (kij) (np.transpose(..., axes=(2,0,1))): (4, 2, 3)
Results match: True


 #### Broadcasting in Einsum





Einsum handles broadcasting automatically when dimensions are missing or are 1, similar to other NumPy functions.

 For example, multiplying a matrix by a vector, where the vector is broadcast across rows or columns.

In [ ]:
A_broadcast_mat = np.array([[1,2,3],[4,5,6]]) # Shape (2,3)
v_broadcast_vec = np.array([10,20,30])        # Shape (3,)
# Multiply A_broadcast_mat by v_broadcast_vec element-wise for each row of A
# 'ij,j->ij': vector j is broadcast to match matrix ij
broadcast_result1 = np.einsum('ij,j->ij', A_broadcast_mat, v_broadcast_vec)
print("Matrix A:\n", A_broadcast_mat)
print("Vector v:", v_broadcast_vec)
print("Broadcast multiply (einsum 'ij,j->ij'):\n", broadcast_result1)
print("Expected (A * v[np.newaxis,:]):\n", A_broadcast_mat * v_broadcast_vec[np.newaxis,:]) # or A_mat * v_vec

v_broadcast_vec2 = np.array([10,20]) # Shape (2,)
# Multiply A_broadcast_mat by v_broadcast_vec2 element-wise for each col of A
# 'ij,i->ij': vector i is broadcast to match matrix ij
broadcast_result2 = np.einsum('ij,i->ij', A_broadcast_mat, v_broadcast_vec2)
print("\nVector v2:", v_broadcast_vec2)
print("Broadcast multiply (einsum 'ij,i->ij'):\n", broadcast_result2)
print("Expected (A * v2[:,np.newaxis]):\n", A_broadcast_mat * v_broadcast_vec2[:,np.newaxis])



Matrix A:
 [[1 2 3]
 [4 5 6]]
Vector v: [10 20 30]
Broadcast multiply (einsum 'ij,j->ij'):
 [[ 10  40  90]
 [ 40 100 180]]
Expected (A * v[np.newaxis,:]):
 [[ 10  40  90]
 [ 40 100 180]]

Vector v2: [10 20]
Broadcast multiply (einsum 'ij,i->ij'):
 [[ 10  20  30]
 [ 80 100 120]]
Expected (A * v2[:,np.newaxis]):
 [[ 10  20  30]
 [ 80 100 120]]


 ## Tensordot





`tensordot` also performs tensor contractions, summing over specified axes.



 ```numpy.tensordot(a, b, axes=2)```



 ```torch.tensordot(a, b, dims=2, out=None)```



 The `axes` (NumPy) or `dims` (PyTorch) parameter is key:

 -  **Integer `N`**: Sums over the last `N` axes of tensor `a` and the first `N` axes of `b`.

 -  **Tuple of two lists `([a_axes_list], [b_axes_list])`**: Sums over specified axes. `a_axes_list[k]` with `b_axes_list[k]`. Resulting tensor has `a`'s remaining axes, then `b`'s remaining axes, in that order.

In [ ]:
# Using PyTorch tensors for tensordot examples as in original files
A_td_torch = torch.rand(2, 3, 4, 5)    # Axes a0,a1,a2,a3
B_td_torch = torch.rand(2, 3, 5, 6)    # Axes b0,b1,b2,b3

# NumPy counterparts for einsum verification
A_td_np = A_td_torch.numpy()
B_td_np = B_td_torch.numpy()

print("Dimensions of A_td_torch (PyTorch):", A_td_torch.shape)
print("Dimensions of B_td_torch (PyTorch):", B_td_torch.shape)


Dimensions of A_td_torch (PyTorch): torch.Size([2, 3, 4, 5])
Dimensions of B_td_torch (PyTorch): torch.Size([2, 3, 5, 6])


 ### Example 1: `tensordot` with specific axes lists





 `torch.tensordot(A_td_torch, B_td_torch, dims=([0,1,3], [0,1,2]))`



 -  `A_td_torch` axes `a0,a1,a2,a3` (2,3,4,5); `B_td_torch` axes `b0,b1,b2,b3` (2,3,5,6).

 -  `dims=([0,1,3], [0,1,2])` contracts:

   -  `a0` (size 2) with `b0` (size 2).

   -  `a1` (size 3) with `b1` (size 3).

   -  `a3` (size 5) with `b2` (size 5).

 -  Remaining from `A_td_torch`: `a2` (size 4). From `B_td_torch`: `b3` (size 6).

 -  Result shape: `(A_td_torch.shape[2], B_td_torch.shape[3])` = `(4,6)`.



 Math: $R_{a_2 b_3} = \sum_{k_0, k_1, k_2} A_{k_0 k_1 a_2 k_2} B_{k_0 k_1 k_2 b_3}$ (where $k_0, k_1, k_2$ are identified common axes).



 `einsum` equivalent for $A_{x_0 x_1 x_2 x_3}$ and $B_{y_0 y_1 y_2 y_3}$:

 Contract $(x_0,y_0)$, $(x_1,y_1)$, $(x_3,y_2)$. Output $x_2, y_3$.

 Let $x_0=s0, x_1=s1, x_3=s2$. Let $y_0=s0, y_1=s1, y_2=s2$.

 $A_{s0 s1 x_2 s2}$ and $B_{s0 s1 s2 y_3}$.

 `'iklm,ikmj->lj'` (if A_axes=[0,1,3] B_axes=[0,1,2])

 A: $s0=i, s1=k, x2=l, s2=m$. B: $s0=i, s1=k, s2=m, y3=j$.

 This corresponds to `einsum('abkc,abcl->kl', A_td_np, B_td_np)` if `k` is A's axis 2 and `l` is B's axis 3.

 Original file 1 used: `einsum('abkc,abcl->kc', A_td_np, B_td_np)` where `k` is A's axis 2, and `c` (representing the last index of B) is B's axis 3. This seems more direct.

In [ ]:
result_td_specific = torch.tensordot(A_td_torch, B_td_torch, dims=([0,1,3], [0,1,2]))
print("Tensordot dims=([0,1,3], [0,1,2]) result shape:", result_td_specific.shape) # Expected (4,6)

einsum_specific_eq = np.einsum('ijAl,ijlk->Ak', A_td_np, B_td_np) # A(i,j,A,l), B(i,j,m,k) -> A,k

print("Einsum 'ijAl,ijlk->Ak' result shape:", einsum_specific_eq.shape)
assert np.allclose(result_td_specific.numpy(), einsum_specific_eq), "Ex1 Tensordot and Einsum results differ!"
print("Tensordot and Einsum results match for Example 1. ✅")


Tensordot dims=([0,1,3], [0,1,2]) result shape: torch.Size([4, 6])
Einsum 'ijAl,ijlk->Ak' result shape: (4, 6)
Tensordot and Einsum results match for Example 1. ✅


 ### Example 2: `tensordot` contracting one pair of axes





 `torch.tensordot(A_td_torch, B_td_torch, dims=([1], [1]))`



 -  Contracts axis `a1` (size 3) of `A_td_torch` with axis `b1` (size 3) of `B_td_torch`.

 -  Remaining from `A_td_torch`: `a0,a2,a3` (sizes 2,4,5). From `B_td_torch`: `b0,b2,b3` (sizes 2,5,6).

 -  Result shape is `(A_td_torch.shape[0], A_td_torch.shape[2], A_td_torch.shape[3], B_td_torch.shape[0], B_td_torch.shape[2], B_td_torch.shape[3])` which is `(2,4,5,2,5,6)`.



 Math: $R_{a_0 a_2 a_3 b_0 b_2 b_3} = \sum_{k} A_{a_0 k a_2 a_3} B_{b_0 k b_2 b_3}$.



 `einsum` equivalent:

 A: $iKlm$, B: $xKy z$ (where $K$ is the summed index, axis 1 for both)

 `einsum('iKlm,xKyz->ilmxz', A_td_np, B_td_np)`

In [ ]:
result_td_ex2 = torch.tensordot(A_td_torch, B_td_torch, dims=([1], [1]))
print("Tensordot dims=([1],[1]) result shape:", result_td_ex2.shape) # Expected (2,4,5,2,5,6)

# A_td_np (i,K,l,m) = (2,3,4,5)
# B_td_np (x,K,y,z) = (2,3,5,6)
# Remaining A: i,l,m. Remaining B: x,y,z. Order: A_rem, B_rem
einsum_eq_ex2 = np.einsum('ijkl,mjno->iklmno', A_td_np, B_td_np) # Corrected output order for B_rem
print("Einsum 'ijkl,mjno->iklmno' result shape:", einsum_eq_ex2.shape)
assert np.allclose(result_td_ex2.numpy(), einsum_eq_ex2), "Ex2 Tensordot and Einsum results differ!"
print("Tensordot and Einsum results match for Example 2. ✅")


Tensordot dims=([1],[1]) result shape: torch.Size([2, 4, 5, 2, 5, 6])
Einsum 'ijkl,mjno->iklmno' result shape: (2, 4, 5, 2, 5, 6)
Tensordot and Einsum results match for Example 2. ✅


 #### Understanding Tensordot Axis Specification



 When we specify `dims=([1], [1])` for `A(a0,a1,a2,a3)` and `B(b0,b1,b2,b3)`:

 - We contract axis `a1` of the first tensor with axis `b1` of the second tensor.

 - The remaining axes from A are `(a0,a2,a3)`.

 - The remaining axes from B are `(b0,b2,b3)`.

 - `tensordot` concatenates these remaining axes in order: `(a0,a2,a3,b0,b2,b3)`.

 - So, the final shape is `(A.shape[0], A.shape[2], A.shape[3], B.shape[0], B.shape[2], B.shape[3])`.

 #### More Tensordot Examples

In [ ]:
A_ex3 = torch.rand(2,3, 4,5) # Ends with 4,5
B_ex3 = torch.rand(4,5, 6,7) # Starts with 4,5
result_td_int_dims = torch.tensordot(A_ex3, B_ex3, dims=2)

print(f"A_ex3 shape: {A_ex3.shape}, B_ex3 shape: {B_ex3.shape}")
print("Tensordot with dims=2 result shape:", result_td_int_dims.shape)

result_td_multi_noncont = torch.tensordot(A_td_torch, B_td_torch, dims=([0,3],[0,2]))
# Remaining A: (a1,a2) -> (3,4)
# Remaining B: (b1,b3) -> (3,6)
# Result shape: (3,4,3,6)
print(f"\nA_td_torch shape: {A_td_torch.shape}, B_td_torch shape: {B_td_torch.shape}")
print("Tensordot with dims=([0,3],[0,2]) result shape:", result_td_multi_noncont.shape)



A_ex3 shape: torch.Size([2, 3, 4, 5]), B_ex3 shape: torch.Size([4, 5, 6, 7])
Tensordot with dims=2 result shape: torch.Size([2, 3, 6, 7])

A_td_torch shape: torch.Size([2, 3, 4, 5]), B_td_torch shape: torch.Size([2, 3, 5, 6])
Tensordot with dims=([0,3],[0,2]) result shape: torch.Size([3, 4, 3, 6])


 Both `einsum` and `tensordot` are powerful. `einsum` is often more flexible for complex ops or specific output axis orders. `tensordot` is handy for standard tensor dot products over sets of axes.

 ## Practical Applications

 ### 1. Batch Processing in Neural Networks (Convolution-like operation)





In deep learning, we often need to process batches of data efficiently. `einsum` can express operations like parts of convolutions.

In [ ]:
# Simulate batch of images and a convolutional kernel
batch_s, channels_in, height_in, width_in = 32, 3, 28, 28
kernel_out_channels, kernel_in_channels, kernel_h, kernel_w = 64, 3, 5, 5

images_batch = np.random.rand(batch_s, channels_in, height_in, width_in)
kernel_conv = np.random.rand(kernel_out_channels, kernel_in_channels, kernel_h, kernel_w)

print("Image batch shape:", images_batch.shape)
print("Kernel shape:", kernel_conv.shape)

image_patch = images_batch[:, :, :kernel_h, :kernel_w] # Shape: (batch_s, channels_in, kernel_h, kernel_w)
print("Image patch shape for operation:", image_patch.shape)

conv_like_op = np.einsum('bcij,ocij->boij', image_patch, kernel_conv)
print("Convolution-like operation (patch) result shape:", conv_like_op.shape)



Image batch shape: (32, 3, 28, 28)
Kernel shape: (64, 3, 5, 5)
Image patch shape for operation: (32, 3, 5, 5)
Convolution-like operation (patch) result shape: (32, 64, 5, 5)


 ### 2. Attention Mechanisms





 Attention mechanisms in transformers use `einsum` (or equivalent batched matrix multiplications) for efficient computation of scores and weighted sums.

In [ ]:
# Simulate query, key, value matrices for attention
seq_len_att, d_model_att = 10, 64
batch_size_att = 8

Q_att = np.random.rand(batch_size_att, seq_len_att, d_model_att)  # Queries (batch, seq_query, dim_model)
K_att = np.random.rand(batch_size_att, seq_len_att, d_model_att)  # Keys   (batch, seq_key,   dim_model)
V_att = np.random.rand(batch_size_att, seq_len_att, d_model_att)  # Values (batch, seq_key,   dim_value usually same as d_model)


attention_scores = np.einsum('bqd,bkd->bqk', Q_att, K_att)
print("Attention scores shape:", attention_scores.shape)

attention_weights_simplified = np.exp(attention_scores) / np.sum(np.exp(attention_scores), axis=-1, keepdims=True)
print("Attention weights shape:", attention_weights_simplified.shape)


attention_output = np.einsum('bqk,bkd->bqd', attention_weights_simplified, V_att)

print("Attention output shape:", attention_output.shape)

Attention scores shape: (8, 10, 10)
Attention weights shape: (8, 10, 10)
Attention output shape: (8, 10, 64)


 ### 3. Statistical Operations





 `einsum` is useful for computing various statistical measures like covariance matrices.

In [ ]:
# Covariance matrix computation
num_samples, num_features = 100, 5
data_stat = np.random.rand(num_samples, num_features)
print(f"Data shape: (samples={num_samples}, features={num_features})")

# Center the data (subtract mean of each feature)
data_centered = data_stat - np.mean(data_stat, axis=0, keepdims=True)

cov_matrix_einsum = np.einsum('sn,sm->nm', data_centered, data_centered) / (num_samples - 1)

# Verification with np.cov (expects features as rows, so transpose data_stat)
cov_matrix_numpy = np.cov(data_stat.T)

print("Covariance matrix shape (einsum):", cov_matrix_einsum.shape)
print("Covariance matrix shape (numpy):", cov_matrix_numpy.shape)
print("Results match numpy.cov:", np.allclose(cov_matrix_einsum, cov_matrix_numpy))



Data shape: (samples=100, features=5)
Covariance matrix shape (einsum): (5, 5)
Covariance matrix shape (numpy): (5, 5)
Results match numpy.cov: True


 ## Performance Tips and Best Practices

 ### 1. Memory Layout and Performance Considerations



 For very large arrays, memory layout (Fortran vs. C order) can impact performance. `einsum` operations are generally efficient. Specific performance can depend on the complexity of the contraction and the underlying BLAS/LAPACK libraries.

In [ ]:
# Example: Matrix multiplication performance
large_A_perf = np.random.rand(1000, 1000)
large_B_perf = np.random.rand(1000, 1000)

print("Performance comparison for large matrix multiplication (1000x1000):")
print("Matrix multiplication - einsum ('ij,jk->ik'):")
%timeit np.einsum('ij,jk->ik', large_A_perf, large_B_perf)

print("Matrix multiplication - einsum with optimization ('ij,jk->ik'):")
%timeit np.einsum('ij,jk->ik', large_A_perf, large_B_perf, optimize=True)
#
print("Matrix multiplication - NumPy @ operator:")
%timeit large_A_perf @ large_B_perf
#
print("Matrix multiplication - np.dot:")
%timeit np.dot(large_A_perf, large_B_perf)

Performance comparison for large matrix multiplication (1000x1000):
Matrix multiplication - einsum ('ij,jk->ik'):
256 ms ± 16.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Matrix multiplication - einsum with optimization ('ij,jk->ik'):
48.4 ms ± 1.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Matrix multiplication - NumPy @ operator:
59.4 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Matrix multiplication - np.dot:
47.7 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


 ### 2. `optimize=True` Parameter in `numpy.einsum`



 NumPy's `einsum` has an `optimize` parameter that can significantly improve performance for complex operations involving three or more operands by finding an optimal contraction order.

In [ ]:
# Complex multi-tensor operation
A_complex_perf = np.random.rand(20, 30, 40)
B_complex_perf = np.random.rand(30, 40, 50)
C_complex_perf = np.random.rand(40, 50, 60)

print("\nPerformance for complex operation ('ijk,jkl,klm->ilm'):")
print("Without optimization (optimize=False):")
%timeit np.einsum('ijk,jkl,klm->ilm', A_complex_perf, B_complex_perf, C_complex_perf, optimize=False)

print("With optimization (optimize=True or default for recent NumPy):")
%timeit np.einsum('ijk,jkl,klm->ilm', A_complex_perf, B_complex_perf, C_complex_perf, optimize=True)
# For recent NumPy versions, `optimize` might default to an intelligent strategy.
# Explicitly setting `optimize=True` (or values like 'greedy', 'optimal') can be beneficial.


Performance for complex operation ('ijk,jkl,klm->ilm'):
Without optimization (optimize=False):
147 ms ± 15.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
With optimization (optimize=True or default for recent NumPy):
5.75 ms ± 99.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


 ### 3. Memory Efficiency



 For very large tensors, consider the memory implications of intermediate arrays that might be created, especially in complex `einsum` expressions if not optimized. `einsum` itself tries to be efficient.

In [ ]:
# Example: Summing along specific axes of a large tensor
large_tensor_mem = np.random.rand(50, 50, 50, 50) # Reduced size for quicker execution

# Sum over axes 1 (j) and 3 (l)
sum_einsum_mem = np.einsum('ijkl->ik', large_tensor_mem)
sum_numpy_mem = np.sum(large_tensor_mem, axis=(1, 3))

print("Shape of sum_einsum_mem ('ijkl->ik'):", sum_einsum_mem.shape)
print("Shape of sum_numpy_mem (sum axis=(1,3)):", sum_numpy_mem.shape)
print("Memory sum operation results match:", np.allclose(sum_einsum_mem, sum_numpy_mem))
# Both methods are generally efficient for this type of sum reduction.


Shape of sum_einsum_mem ('ijkl->ik'): (50, 50)
Shape of sum_numpy_mem (sum axis=(1,3)): (50, 50)
Memory sum operation results match: True


 ## Common Patterns



 Here are some frequently used `einsum` patterns:

In [ ]:
# Pattern collection
common_einsum_patterns = {
    "Matrix transpose": "'ij->ji'",
    "Batch matrix multiplication": "'bij,bjk->bik'",
    "Vector dot product": "'i,i->'",
    "Vector outer product": "'i,j->ij'",
    "Matrix trace (sum of diagonal)": "'ii->'",
    "Matrix diagonal extraction": "'ii->i'",
    "Sum all elements (tensor)": "'...->'",
    "Sum all elements (matrix)": "'ij->'",
    "Sum matrix along axis 0 (collapse rows)": "'ij->j'",
    "Sum matrix along axis 1 (collapse columns)": "'ij->i'",
    "Element-wise matrix multiplication": "'ij,ij->ij'",
    "Matrix times vector (broadcast over rows)": "'ij,j->ij'", # Vector 'j' applied to each row 'i'
    "Vector times matrix (broadcast over columns)": "'ij,i->ij'", # Vector 'i' applied to each column 'j'
    "Quadratic form (vector-matrix-vector)": "'i,ij,j->'", # u^T A v
    "Batch diagonal extraction": "'...ii->...i'",
    "Batch trace": "'...ii->...'",
}

print("Common Einsum Patterns:")
for name, pattern_str in common_einsum_patterns.items():
    print(f"{name:40}: {pattern_str}")


Common Einsum Patterns:
Matrix transpose                        : 'ij->ji'
Batch matrix multiplication             : 'bij,bjk->bik'
Vector dot product                      : 'i,i->'
Vector outer product                    : 'i,j->ij'
Matrix trace (sum of diagonal)          : 'ii->'
Matrix diagonal extraction              : 'ii->i'
Sum all elements (tensor)               : '...->'
Sum all elements (matrix)               : 'ij->'
Sum matrix along axis 0 (collapse rows) : 'ij->j'
Sum matrix along axis 1 (collapse columns): 'ij->i'
Element-wise matrix multiplication      : 'ij,ij->ij'
Matrix times vector (broadcast over rows): 'ij,j->ij'
Vector times matrix (broadcast over columns): 'ij,i->ij'
Quadratic form (vector-matrix-vector)   : 'i,ij,j->'
Batch diagonal extraction               : '...ii->...i'
Batch trace                             : '...ii->...'


 ## Conclusion



 This notebook demonstrated the power and flexibility of `einsum` and `tensordot` operations:



 ### Key Takeaways:



 1.  **Concise Notation**: `einsum` provides a powerful and concise domain-specific language for tensor operations using Einstein summation convention.

 2.  **Versatility**: Both functions can perform a wide array of operations including, but not limited to, dot products, outer products, transpositions, permutations, and complex tensor contractions.

 3.  **Performance**: `einsum` can be highly performant, especially with the `optimize` flag for complex contractions, often matching or exceeding manually chained NumPy operations. `tensordot` is also optimized for its specific task.

 4.  **Readability**: While initially cryptic, `einsum` strings can become very readable for those familiar with the notation, clearly expressing the intent of tensor manipulations.

 5.  **Framework Support**: Available in both NumPy for CPU-bound tasks and PyTorch (often with GPU acceleration) for deep learning.



 ### When to Use Each:



 -   **`einsum`**:

     -   For operations requiring flexible control over which indices are summed, kept, or reordered.

     -   When performing multiple contractions or a sequence of operations that can be expressed in a single `einsum` string (often more efficient due to optimization).

     -   For operations that are hard to express concisely with standard matrix/tensor multiplication and transpose/summation routines (e.g., trace of a batch of matrices, specific diagonal extractions, attention-like mechanisms).

 -   **`tensordot`**:

     -   When you need to contract specific pairs of axes between two tensors.

     -   It's a good general-purpose tensor dot product when the `axes` argument clearly defines the contraction.

     -   Can be more straightforward than `einsum` if you think in terms of "sum over these axes from tensor A and these axes from tensor B."

 -   **Standard operators (`@`, `np.dot`, `np.multiply`, `np.sum`, `.T`, etc.)**:

     -   For very common and simple operations like element-wise multiplication, matrix multiplication of two matrices, or simple sums/transpositions. These are often the most readable for universally understood operations and are highly optimized.



 ### Best Practices:



 1.  **Clarity**: Add comments to complex `einsum` strings explaining the index notation and the operation being performed.

 2.  **Optimization**: For `numpy.einsum` involving three or more tensors, explore the `optimize=True` (or more specific strategies like 'greedy' or 'optimal') argument for potential speedups.

 3.  **Profiling**: For performance-critical sections, profile `einsum` against alternative formulations (e.g., using `tensordot` or a sequence of standard NumPy/PyTorch ops).

 4.  **Readability vs. Conciseness**: Balance the conciseness of `einsum` with the readability of more verbose standard operations, especially for simpler tasks where `einsum` might be overkill.

 5.  **Start Simple**: If new to `einsum`, start by reformulating simple operations (like dot products, transposes) and gradually move to more complex ones.



 Mastering `einsum` and `tensordot` can significantly enhance your ability to implement complex algorithms in scientific computing and machine learning efficiently and effectively.